In [38]:
import os
from pyspark.sql.functions import col
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,555 B in 1s (4,751 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [39]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [40]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [41]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=",", header=True)
df.createOrReplaceTempView('home_sales')
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [42]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places
averages = df.groupby('date_built').agg({'price': 'mean'}).withColumnRenamed('avg(price)', 'average_price').orderBy(df['date_built'].asc())
averages.show()

+----------+------------------+
|date_built|     average_price|
+----------+------------------+
|      2010| 309751.1693922382|
|      2011| 314804.3172613308|
|      2012|314072.76632384595|
|      2013| 317597.6014687882|
|      2014|316395.73264472734|
|      2015| 315322.6367702448|
|      2016| 315471.5675872093|
|      2017| 315799.0613641728|
+----------+------------------+



In [43]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
filtered_averages = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3)).groupby('date_built').agg({'price': 'mean'}).withColumnRenamed('avg(price)', 'average_price') \
                                                                                                                          .orderBy(df['date_built'].asc())
filtered_averages.show()


+----------+------------------+
|date_built|     average_price|
+----------+------------------+
|      2010|  292859.615942029|
|      2011|291117.46706586826|
|      2012| 293683.1872074883|
|      2013|295962.27145085804|
|      2014| 290852.2661870504|
|      2015| 288770.2966101695|
|      2016|  290555.073964497|
|      2017| 292676.7887740029|
+----------+------------------+



In [44]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
filtered_averages = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3)).filter(df['floors'] == 2).filter(df['sqft_living'] >= 2000) \
                                                                                                       .groupby('date_built').agg({'price': 'mean'}).withColumnRenamed('avg(price)', 'average_price').orderBy(df['date_built'].asc())
filtered_averages.show()

+----------+------------------+
|date_built|     average_price|
+----------+------------------+
|      2010| 285010.2215909091|
|      2011| 276553.8128654971|
|      2012|307539.97402597405|
|      2013|      303676.79375|
|      2014| 298264.7183908046|
|      2015| 297609.9679144385|
|      2016| 293965.1046511628|
|      2017|280317.57692307694|
+----------+------------------+



In [45]:
from typing import ValuesView
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_averages = df.groupby('view').agg({'price': 'mean'}).withColumnRenamed('avg(price)', 'average_price').orderBy(col('view').cast('int').desc())
view_averages.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     average_price|
+----+------------------+
| 100|         1026669.5|
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 1.0307505130767822 seconds ---


In [47]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [48]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [49]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("path/to/save/home_sales")

In [52]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("path/to/save/home_sales")

In [53]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_temp")

In [56]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

parquet_averages = parquet_df.filter(parquet_df['price'] >= 350000).groupby('view').agg({'price': 'mean'}).withColumnRenamed('avg(price)', 'average_price').orderBy(col('view').cast('int').desc())
parquet_averages.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     average_price|
+----+------------------+
| 100|         1026669.5|
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 1.0069961547851562 seconds ---


In [57]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [58]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False